# Create Fake User

- Config object


In [ ]:
#Imports
from emission.simulation.client import EmissionFakeDataGenerator
from emission.simulation.fake_user import FakeUser
import emission.storage.timeseries.cache_series as estcs
import requests
import numpy as np
import pandas as pd
from time import sleep

In [ ]:
#Step1 : specify a config object for user
client_config = {
    'emission_server_base_url': 'http://localhost:8080',
    'register_user_endpoint': '/profile/create',
    'user_cache_endpoint': '/usercache/put'
}

user_config = {
            "email" : 'fake_user_129',
    
            "locations" :
            [
               {
                    'label': 'home',
                    'coordinate': [37.77264255,-122.399714854263]
                },

                {
                    'label': 'work',
                    'coordinate': [37.42870635,-122.140926605802]
                },
                {
                    'label': 'family',
                    'coordinate': [37.87119, -122.27388]
                }
            ],
            "transition_probabilities":
            [
                np.random.dirichlet(np.ones(3), size=1)[0],
                np.random.dirichlet(np.ones(3), size=1)[0],
                np.random.dirichlet(np.ones(3), size=1)[0]
            ],
    
            "modes" : 
            {
                "CAR" : [['home', 'family']],
                "TRANSIT" : [['home', 'work'], ['work', 'home']]  
            },

            "default_mode": "CAR",
            "initial_state" : "home",
            "radius" : ".1"
        }


In [ ]:
np.random.dirichlet(np.ones(3), size=1)[0]

In [ ]:
client = EmissionFakeDataGenerator(client_config)

In [ ]:
fake_user = client.create_fake_user(user_config)

## Simulate User Behavior

In [ ]:
measurements = []
for _ in range(4):
    """
    
    """
    temp = fake_user.take_trip()
    print('# of location measurements:', len(temp))
    measurements.append(temp)
    #try:
        #respons = requests.post('server_address', payload={'data':measurements})
    #except:
        
    #sleep(2)

print('Path:',fake_user._path)
#Run pipeline


# Push data to server

In [ ]:
len(fake_user._measurements_cache)

In [ ]:
fake_user.sync_data_to_server()

In [ ]:
len(fake_user._measurements_cache)

In [ ]:
location_data = [entry.data for sublist in measurements for entry in sublist if entry.metadata.key == 'background/filtered_location']
#meta_data = [entry.metadata for sublist in measurements for entry in sublist if entry.metadata.key == 'background/filtered_location']

In [ ]:
pd.DataFrame(location_data)

In [ ]:
# Run pipeline manually 
#(tsdb_count, ucdb_count) = estcs.insert_entries(override_uuid, measurements)
#print("Finished loading %d entries into the usercache and %d entries into the timeseries" %
#(ucdb_count, tsdb_count))

## Run intake pipeline

In [ ]:
email = fake_user._email

In [ ]:
!source activate emission
!./e-mission-py.bash bin/debug/intake_single_user.py -e 'fake_user_129'

## Display data
- Show trip entries
- Show trips on the map 

In [ ]:
from uuid import UUID
import arrow
fake_user_id = UUID(fake_user._uuid)

In [ ]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [ ]:
print(fake_user_id)
ts = esta.TimeSeries.get_time_series(fake_user_id)

In [ ]:
# Raw trips
rt_df = ts.get_data_df('segmentation/raw_trip', time_query=None)
raw_trips = rt_df[["start_loc", "end_loc", "start_ts", "end_ts"]]
raw_trips['start_time'] = raw_trips["start_ts"].apply(lambda x : arrow.get(x).format())
raw_trips['end_time'] = raw_trips["end_ts"].apply(lambda x : arrow.get(x).format())
raw_trips['end_coord'] = raw_trips["end_loc"].apply(lambda x : dict(x)['coordinates'])
raw_trips['start_coord'] = raw_trips["start_loc"].apply(lambda x : dict(x)['coordinates'])

In [ ]:
raw_trips[['start_time', 'end_time',"start_coord", "end_coord"]]

In [ ]:
# Get all cleaned trips for the first user
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=None)

In [ ]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", fake_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)